# 1. Newsgroup dataset

The 20 Newsgroups data set is a collection of approximately 20,000 newsgroup documents, partitioned (nearly) evenly across 20 different newsgroups. In this assignment we are going to use a **subset** of this dataset to learn a bit about unsupervised learning methods in machine learning. To load this dataset we are going to use scikit-learn library which you have worked with a couple of times through semester.

# 2. Loading data and preprocessing

To ensure that your results are reproducable make sure to set the seed value.

In [10]:
import numpy as np
import os
import sys

print(os.path.dirname(sys.executable))

# os.chdir('C:\\Users\\17164\Desktop\Python\\assignment4')

SEED = 123
np.random.seed(SEED)

D:\


## 2.1. downloading and loading data

You can download both training set and test set separately using sklearn apis. [Here](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) you can find sample codes for loading dataset.

In [3]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

# YOUR CODE HERE
newsgroups_train = fetch_20newsgroups(subset='train',remove=('headers,footers','quotes'))
df_221 = pd.DataFrame(newsgroups_train.target_names)
newsgroups_test = fetch_20newsgroups(subset='test',remove=('headers,footers','quotes'),return_X_y=True)
newsgroups_train = fetch_20newsgroups(subset='train',remove=('headers,footers','quotes'),return_X_y=True)
newsgroups_train_df = pd.DataFrame(newsgroups_train).T
newsgroups_train_df.columns = ['newstitle','target']
newsgroups_test_df = pd.DataFrame(newsgroups_test).T
newsgroups_test_df.columns = ['newstitle','target']

## 2.2 filtering target classes

Explore dataset and get familiar with it. Then, implement `filter_split_x_y` function which extracts `selected_targets` classes from data and splits the data into two lists of X and ys. Please don't change `selected_targets` list is this cell.


In [4]:

selected_targets = [1, 7, 10, 13, 15, 16, 17]

def filter_split_X_y(raw_data, selected_targets):
    """
    This function inputs a newsgroup dataset and filters it based on your selected labels
    then returns two lists of datapoints and labels separately
    
    """
    result_data = raw_data[raw_data['target'].isin(selected_targets)]
    return result_data['newstitle'],result_data['target']
    # YOUR CODE HERE
    

train_x, train_y = filter_split_X_y(newsgroups_train_df, selected_targets)
test_x, test_y = filter_split_X_y(newsgroups_test_df, selected_targets)


**2.2.1.** Print the name of classes in your training set along with `selected_targets` you can use `target_names` attribute of `newsgroups_train`. Make sure you include this output in your PDF report.

In [5]:
df_221_ans = df_221.iloc[selected_targets]
df_221_ans.columns=['target_names']
df_221_ans.index.name = 'target'
df_221_ans.to_csv('df_221.csv')

## 2.3. vectorizing documents

There are several ways that one can use to vectorize a whole document. In this programming assignment we are going to use **TF-IDF** method. Please go ahead and watch [this short video](https://www.youtube.com/watch?v=D2V1okCEsiE) to understand how tf-idf works and then answer the following questions.

**2.3.1.** What does TF-IDF stand for?<br>
**2.3.2.** Why don't we only use term frequency of the words in a document as its feature vector? what is the benefit of adding inverse document frequency?<br>
**2.3.3.** Calculate the tf-idf vectors of the following two documents, assuming this is the entire corpus:
![Documents](tfidf.png)


Now we are going to use scikit-learn to calculate the tf-idf vectors of each document in our twenty newgroups dataset. Write a function that accepts both `train_x` and `test_x` and returns the tf-idf vectors of them in numpy format.

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

def vectorize_dataset(train_x, test_x):
    """
    returns vectorized numpy array of training and test set
    """
    vectorizer = TfidfVectorizer()
    train_vec = vectorizer.fit_transform(train_x)
    test_vec = vectorizer.transform(test_x)
    return train_vec.toarray(),test_vec.toarray()
    # YOUR CODE HERE
    
train_vec, test_vec = vectorize_dataset(train_x, test_x)
print(type(train_vec),type(test_vec))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


# 3. dimension reduction

In the previous section we built features vectors for each of the documents. However, these feature vectors are highly sparse and are not easy to cluster or visualize.

## 3.1. Sparsity

Answer these questions to understand the sparsity of these vectors:

**3.1.1** Count the number of non-zeros in each row of the `train_vec` matrix.<br>
**3.1.2** What is the average number non zero elements in each row? <br>
**3.1.3** On average what percentage of elements in each row have non-zero elements?<br>

In [7]:

train_vec_df = pd.DataFrame(train_vec)
print(train_vec_df.shape)
# test_vec_df = pd.DataFrame(test_vec)
count_non_zero = (train_vec_df!=0).astype(int).sum(axis=1)

#3.1.1 count_non_zero
count_non_zero_list = count_non_zero.tolist()
# print(count_non_zero_list)
#3.1.2 count_non_zero_avg
count_non_zero_avg = sum(count_non_zero_list)/len(count_non_zero_list)
print(count_non_zero_avg)
# count_non_zero_list_avg = count_non_zero_list.
count_non_zero_perc = (train_vec_df!=0).astype(int).mean(axis=1)
count_non_zero_perc = count_non_zero_perc*100
#3.1.3 average percentage of elements
count_non_zero_perc_list = count_non_zero_perc.tolist()
# print(count_non_zero_list[0]/52057)
# print(count_non_zero_perc_list[0])

(4081, 52057)
135.02597402597402


## 3.2. SVD

Use `TruncatedSVD` module in scikit-learn to perform SVD on the dataset. Reduce your dimensions to 3 and perfomr SVD for 100 iterations (**note: the number of iterations parameter is a special component for an alternative way to solve SVD that we did not study in class. You do not need to know what it means**). Make sure to use the random seed given at the beginning of the notebook.

**Note: you should only fit your SVD/UMAP models on train data**

**3.2.1.** What portion of the variance in your dataset is explained by each of the SVD dimensions?

In [8]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

def fit_and_transform_svd(train_vec, test_vec):
    """
    trains a svd model and generate reduced dimension vectros from training and test dataset
    returns: 2 numpy arrays of size <number of documents>*3
    """
    # YOUR CODE
    train_vec_csr = csr_matrix(train_vec)
    test_vec_csr = csr_matrix(test_vec)
    svd = TruncatedSVD(n_components=3, n_iter=100, random_state=SEED)
    train_svd = svd.fit_transform(train_vec_csr)
    print(svd.explained_variance_ratio_)
    test_svd = svd.transform(test_vec_csr)
    return train_svd,test_svd
    


# print(type(train_vec),type(test_vec))    
train_svd, test_svd = fit_and_transform_svd(train_vec, test_vec)

[0.01537037 0.00908879 0.00573035]


## 3.3 UMAP

UMAP is another dimensionality reduction method that works in a more complex way. [This video](https://www.youtube.com/watch?v=6BPl81wGGP8) explains on a high level how it performs dimension reduction.
Apply UMAP and get the embeddings for both training and test datasets. Use the previously given `random_state` and reduce dimensions to 3. Also use **cosine similarity** as the similarity metric of UMAP:

- n_components=3
- metric='cosine'
- random_state=SEED
- low_memory=True

In [9]:
import umap

def fit_and_transform_umap(train_vec, test_vec):
    # YOUR CODE
    train_vec_csr = csr_matrix(train_vec)
    test_vec_csr = csr_matrix(test_vec)
    reducer = umap.UMAP(n_components=3,metric='cosine',random_state=SEED,low_memory=True)
    train_umap = reducer.fit_transform(train_vec_csr)
    test_umap = reducer.transform(test_vec_csr)
    return train_umap,test_umap
        
    
    
train_umap, test_umap = fit_and_transform_umap(train_vec, test_vec)

AttributeError: module 'umap' has no attribute 'UMAP'

## 3.4. visualization

install `babyplots` library which provides some nice 3D visaulizations. Then, visualize both of embeddings (SVD and UMAP) of the **training data** using the given function. **make sure you put a screenshot of the visualizations inside your write up**. You can use it to explain your answers to the questions.

<span style="color:red"> Please don't use babyplot inside <b>jupyter lab</b> as it is still not fully supported and produces some errors </span>

Hint: you need to run the function in a separate cell to get the visualization

**3.4.1.** Based on your observation, what is the difference between SVD and UMAP embeddings? 1-2 sentences should suffice.<br>
**3.4.2.** Which one do you prefer to use for a classification task? why? 1-2 sentences should suffice <br>

In [ ]:
from babyplots import Babyplot

def visualize(datapoints, labels):
    bp = Babyplot(background_color="#ffffffff", turntable=True)
    bp.add_plot(
        datapoints,
        "pointCloud",
        "categories",
        np.array(labels).tolist(),
        {
            "colorScale": "Dark2", 
            "showLegend": True
        }
    )
    return bp

In [ ]:
visualize(train_umap, train_y)

In [ ]:
visualize(train_svd, train_y)

# 4. Kmeans clustering



In this part we are going to perform KMeans clustering on the generated embeddings of our training dataset. One major challenge with KMeans is to find the optimum number of clusters to use for clustering. Here, based on our training data, we know that there are 7 clusters in the dataset. However, we want to find the number of optimum clusters solely based on our data. In this part we are going to use [`average silhouette coefficient`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html#sklearn.metrics.silhouette_score) to evaluate our clustering


## 4.1. clustering and evaluation

**4.1.1** What is the range of possible values of silhouette coefficients? <br>
**4.1.2** Describe what a silhouette score of -1 and 1 mean?<br>
**4.1.3.** Use `silhouette score` and `KMeans` from sklearn library to find the optimum number of clusters in your `train_umap`. Don't forget to use `SEED` as your kmeans `random_seed`. In order to do this try different values of cluster numbers from 5 to 20. Choose the one that results in the best score. <br>
**4.1.4.** Plot silhouette score for different values of `n_clusters` (a plot with `n_clusters` on the x-axis and silhouette score on the y-axis) and find the best value for `n_cluster`. Don't forget to put the plot in your report.

In [ ]:
%%time

from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

def get_clustering_scores(train_vectors) -> KMeans:
    """
    This function calculates KMeans model for different values of n_cluster and calculates silhouette score
    for each of them. Then it returns a list of silhouette scores
    RETURNS: 
        list of scores
    """
    # YOUR CODE
    


clustering_scores = get_clustering_scores(train_umap)

## plot scores
# YOUR CODE

## Find best number of clusters
# YOUR CODE

## 4.2. Making a kmeans classifier

Now that we have the best number of clusters, run KMeans again with the best number of clusters associated with the best sihouette score (on `train_umap`). 

After running KMeans you get some number of clusters but each of them have some cluster label that does not necessarily match the trainig labels. Try to implement `get_cluster_mapping` function that inputs your KMeans model and the training labels and returns a dictionary that maps each of the cluster labels to one of the training labels. For example your cluster 0 might correspond to training label of 17.

Hint: In your `get_cluster_mapping` function, find the training label of the majority of data points in each cluster. For example, if your cluster 0 contains 100 data points of label 12 and 5 data points of label 15, the majority of data points in cluster 0 come from label 12. Therefore, your output dictionary should contain the mapping `0: 12`

**4.2.1** show your mapping (resulted dictionary) inside your project report.


In [ ]:

# Run KMeans again with best parameter you calculated in the previous section
# YOUR CODE HERE

###


def get_cluster_mapping(clustering: KMeans, original_labels):
    """
    input: a clustering and original labels of the data
    returns: a dictionary that maps each cluster number to an original label
    """
    # YOUR CODE HERE
    

# Feed the generated kmeans clustering and your training labels to the following function
cluster_mapping = get_cluster_mapping(clustering, train_y)
print(cluster_mapping)

## 4.3. Analyzing clusters

**4.3.1.** Are there any two clusters in your clustering output with the same training label (for example, are there two clusters which both have same training label)? Use your visualizations and describe why?<br>
**4.3.2.** Write the function bellow that returns nearest samples to a cluster center. Use this function and explain why there are overlaps in your labels? <br>
**4.3.3.** Can you infere the overlapping label(s) by checking out most central samples? check with original labels.<br>

In [ ]:
from scipy.spatial.distance import cdist

def most_central_samples(clustering: KMeans, cluster_id, k=3):
    """
    returns the text of k most central samples in the specified cluster_id
    """
    # YOUR CODE
    


## 4.4. evaluate your kmeans model on test dataset

**4.4.1.** Using the generated mapping, and your clustering model, predict the labels of test dataset (you can use the embeddings of test data that you generated by umap `test_umap`) <br>
**4.4.2.** Calculate the accuracy of model <br>
**4.4.3.** Calculate both micro and macro values of precision, recall and F1 score <br>

In [ ]:
# YOUR CODE HERE


# **574 Only** 5.1 KNN classification

Using sklearn `KneighborsClassifier`, classify news data. Then, evaluate your model using the test set.

**5.1.1.** Train two seperate KNN models on both SVD and UMAP embeddings. Use `n_neighbors=100`. <br>
**5.1.2.** Evaluate your model on test datas (`test_umap` and `test_svd`). Which model performs better? Why? <br>
**5.1.3.** Calculate macro and micro precision recall and fscore for `test_umap`. Which one of the two do you prefer for evaluating your model? why? <br>
**5.1.4.** Shortly describe why the two sets of values (macro and micro) are so similar in this case. <br>

In [ ]:
#YOUR CODE
